In [ ]:
!pip install pymorphy2
!pip install detokenize

In [ ]:
POS_TAGS = [
    "adj",
    "adv",
    "intj",
    "noun",
    "propn",
    "verb",
    "adp",
    "aux",
    "cconj",
    "det",
    "num",
    "part",
    "pron",
    "sconj",
    "x"
]

In [ ]:
import csv
import pymorphy2
from detokenize.detokenizer import detokenize

morph = pymorphy2.MorphAnalyzer()


class SearchEngineCSV:
    def __init__(self, csv_file):
        self.sentences = []
        self.tokens = []
        self.lemmas = []
        self.pos_tags = []
        self.chapters = []

        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                self.sentences.append(row[0])
                self.tokens.append([token.lower() for token in row[1].split(';')])  # Токены в нижнем регистре
                self.lemmas.append([lemma.lower() for lemma in row[2].split(';')])  # Леммы в нижнем регистре
                self.pos_tags.append(row[3].split(';'))
                self.chapters.append(row[4])

    def match_token(self, token, word, lemma, pos):
        token = token.lower()  # Приведение токена к нижнему регистру
        if '"' in token:
            return token.strip('"') == word

        if '+' in token:
            query_word, query_pos = token.split('+')
            return lemma.lower() == query_word.lower() and pos.lower() == query_pos.lower()

        if token in POS_TAGS:
            return pos.lower() == token

        return lemma == morph.parse(token)[0].normal_form

    def match_sequence(self, query_tokens, words, lemmas, pos_tags):
        if len(query_tokens)>2 and '"' in query_tokens[2]:
            query_tokens[1] = query_tokens[1] + '"'
        for j, query_token in enumerate(query_tokens):
            query_token = query_token.lower()  # Приведение токена запроса к нижнему регистру
            if query_token in POS_TAGS:
                if pos_tags[j].lower() != query_token:
                    return False
            else:
                if not self.match_token(query_token, words[j], lemmas[j], pos_tags[j]):
                    return False
        return True

    def search(self, query):
        query_tokens = query.lower().split()  # Приведение запроса к нижнему регистру

        matches = []
        for idx, sentence in enumerate(self.sentences):
            words = self.tokens[idx]
            lemmas = self.lemmas[idx]
            pos_tags = self.pos_tags[idx]
            if len(words) != len(lemmas) or len(words) != len(pos_tags):  # проверка, что количество токенов, лемм и тегов частей речи одинаковое
                continue

            # Поиск последовательности токенов в предложении
            for i in range(len(words) - len(query_tokens) + 1):
                formatted_sentence = ""
                matches_in_sent = []
                # Проверка последовательности для n-граммы
                if self.match_sequence(query_tokens, words[i:i+len(query_tokens)], lemmas[i:i+len(query_tokens)], pos_tags[i:i+len(query_tokens)]):
                    # Форматируем предложение с источником
                    source_info = f"[{self.chapters[idx]}]"
                    if formatted_sentence == "":
                        formatted_sentence = f"{sentence} {source_info}"
                    matches_in_sent.append((i, i + len(query_tokens)))  # информация о том, какие слова - ответ на запрос (номер первого слова последовательности и первого слова после неё)
                if formatted_sentence != "":
                    matches.append([formatted_sentence, matches_in_sent, idx])  # строка "текст предложения [номер главы]", список индексов слов, соответствующих запросу, и номер предложения

        return matches

    def frequency(self, query):  # функция, определяющая частотность, принимает результаты поиска
        search_results = self.search(query)
        freq_list = {}  # вариант ответа на запрос : его частотность
        for results_elem in search_results:
            entry_indexes = results_elem[1]  # каких номерах слова, соответствующие самому запросу (номер первого слова и следующего после последнего слова)
            sent_num = results_elem[2]
            for i in entry_indexes:
                entry_tokens = self.tokens[sent_num][i[0]:i[1]]
                entry = detokenize(entry_tokens).replace('- ', '-')
                freq_list[entry] = freq_list.get(entry, 0) + 1
        freq_list_array = list(freq_list.items())
        freq_list_sorted =  sorted(freq_list_array, key=lambda x: x[1], reverse=True)
        return freq_list_sorted


# Пример использования
csv_file = '/content/corpus_data_with_labels.csv'  # Путь к файлу
search_engine = SearchEngineCSV(csv_file)

# Код для запроса и вывода результатов
print('Введите запрос:')
for_search = input()
results = search_engine.search(for_search)
print(f'Количество примеров: {len(results)}')
set_results = {}
for result in results:
    print(result)

Выходные данные были обрезаны до нескольких последних строк (5000).
['К счастию, по причине неудачной охоты, наши кони не были измучены: они рвались из-под седла, и с каждым мгновением мы были всё ближе и ближе... И наконец я узнал Казбича, только не мог разобрать, что такое он держал перед собою. [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', [(7, 8)], 330]
['К счастию, по причине неудачной охоты, наши кони не были измучены: они рвались из-под седла, и с каждым мгновением мы были всё ближе и ближе... И наконец я узнал Казбича, только не мог разобрать, что такое он держал перед собою. [Герой нашего времени. Часть 1. Бэла. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-02.html]', [(14, 15)], 330]
['К счастию, по причине неудачной охоты, наши кони не были измучены: они рвались из-под седла, и с каждым мгновением мы были всё ближе и ближе... И наконец я узнал Казбича, только не мог разобрать, что такое он держал перед собою. [Ге

In [ ]:
# Примеры запросов
print(search_engine.search('сказать'))  # Любая форма слова "сказать"
print(search_engine.search('"меня"'))  # Точная форма "меня"
print(search_engine.search('человек+noun'))  # Лемма "человек" с POS-тегом "S"
print(search_engine.search('NOUN ADV VERB'))  # Существительное, глагол, наречие в последовательности

[['Вы мне опять скажете, что человек не может быть так дурен, а я вам скажу, что ежели вы верили возможности существования всех трагических и романтических злодеев, отчего же вы не веруете в действительность Печорина? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', [(3, 4)], 12], ['Вы мне опять скажете, что человек не может быть так дурен, а я вам скажу, что ежели вы верили возможности существования всех трагических и романтических злодеев, отчего же вы не веруете в действительность Печорина? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', [(14, 15)], 12], ['Вы скажете, что нравственность от этого не выигрывает? [Герой нашего времени. Предисловие. - rvb.ru - https://rvb.ru/19vek/lermontov/lp/text/01-01.html]', [(1, 2)], 15], ['— Скажите, пожалуйста, отчего это вашу тяжелую тележку четыре быка тащат шутя, а мою пустую шесть скотов едва подвигают с помощию этих осетин? [Герой нашего вр

In [ ]:
# Примеры частотностей
print(search_engine.frequency('NOUN')[:5])
print(search_engine.frequency('я')[:5])
print(search_engine.frequency('знать+VERB')[:5])
print(search_engine.frequency('NOUN ADV VERB')[:5])

[('глаза', 49), ('раз', 45), ('руку', 33), ('княжна', 29), ('человек', 27)]
[('я', 887), ('мне', 255), ('меня', 211), ('мной', 16), ('мною', 14)]
[('знаю', 27), ('знает', 20), ('знаете', 16), ('знаешь', 9), ('знал', 8)]
[('извозчик неутомимо погонял', 1), ('речкой шумно вырывающейся', 1), ('скотов едва подвигают', 1), ('осетины шумно обступили', 1), ('хлеба по-русски назвать', 1)]
